In [1]:
# from datasets import load_dataset

# eli5 = load_dataset("eli5", split="train_asks[:5000]")
# eli5 = eli5.train_test_split(test_size=0.2)

# eli5 = eli5.flatten()
# eli5["train"][0]

import json
import torch
from datasets import load_dataset, Dataset, load_from_disk

fin = open("datasets/parametric_equations_sbr.json", 'r')
lines = fin.readlines()
sentences = []
for line in lines:
    data = json.loads(line)
    sentences.append(data['rounded_regressed'] + ' entail ' + data['original'] + 'end')
fin.close()

ds = Dataset.from_dict({'eq_pair': sentences})
train_ds = ds.train_test_split(test_size=0.08) 

train_ds['train'][1]

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'eq_pair': '-2.24*t + 4.47 - 0.e-2/(3.13*t + 89.4*Abs(4.0*t - 7.62) + 0.45) entail -sqrt(5)*t + 2*sqrt(5)end'}

In [2]:
from transformers import AutoTokenizer

CONTEXT_LENGTH = 256

tokenizer = AutoTokenizer.from_pretrained("xhyi/PT_GPTNEO350_ATG") #"EleutherAI/gpt-neo-125m")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    return tokenizer(examples["eq_pair"], padding='max_length', truncation=True, max_length=CONTEXT_LENGTH, return_tensors="pt")

tokenized_ds = train_ds.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=train_ds["train"].column_names,
)

def preprocess_function(examples):
    examples["labels"] = examples["input_ids"].copy()
    return examples

lm_dataset = tokenized_ds.map(preprocess_function, batched=True, num_proc=1)

from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/home/mcwave/anaconda3/envs/symbolic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'train_ds' is not defined

In [3]:
lm_dataset['test'][1]

{'input_ids': [16,
  13,
  15,
  9,
  83,
  1174,
  17,
  1343,
  860,
  13,
  2091,
  9,
  83,
  1343,
  2310,
  13,
  3695,
  39793,
  256,
  1174,
  17,
  1343,
  2579,
  9,
  83,
  14,
  18,
  1343,
  2310,
  13,
  3324,
  3324,
  3324,
  3324,
  3324,
  39761,
  437,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256

In [4]:
for i in range(len(lm_dataset['train'])):
    if len(lm_dataset['train'][i]['input_ids']) != 256:
        print(i, len(lm_dataset['train'][i]['input_ids']))
    if len(lm_dataset['train'][i]['labels']) != 256:
        print(i, len(lm_dataset['train'][i]['labels']))

In [5]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("xhyi/PT_GPTNEO350_ATG") #"EleutherAI/gpt-neo-125m")

training_args = TrainingArguments(
    output_dir="datasets/normalize_symbolic_regression_results_20231215",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    fp16=True,
    save_steps=500,
    eval_steps=500,
    logging_steps=500,
    save_total_limit=2,
    push_to_hub=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,0.847400,0.655413
1000,0.587500,0.551789
1500,0.494300,0.509035
2000,0.455600,0.482812
2500,0.417700,0.468753
3000,0.393600,0.454464
3500,0.378600,0.461721
4000,0.353700,0.455750
4500,0.342100,0.457754
5000,0.311000,0.456163


TrainOutput(global_step=5500, training_loss=0.4453062771883878, metrics={'train_runtime': 1414.4042, 'train_samples_per_second': 31.087, 'train_steps_per_second': 3.889, 'total_flos': 2.041250533343232e+16, 'train_loss': 0.4453062771883878, 'epoch': 5.0})

In [10]:
model.save_pretrained("datasets/normalize_symbolic_regression_results_20231215/gptneo-350m-5500.model")

In [3]:
# Encode some input text
prompt = "0.33*t**3 - 1.0*t**2 + 2.0*t entail"
input_ids = tokenizer.encode(prompt, return_tensors='pt').cuda()

# Generate text
output = model.generate(input_ids, max_length=50, num_return_sequences=1, temperature=0.7)

# Decode and print the output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

NameError: name 'model' is not defined